In [1]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForCausalLM, Trainer, TrainingArguments
torch.cuda.empty_cache()

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = load_dataset("code_search_net", "python",cache_dir="./Datasets")

In [4]:
dataset.column_names

{'train': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'test': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'validation': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url']}

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base", cache_dir="./Models")
model = RobertaForCausalLM.from_pretrained("microsoft/codebert-base", cache_dir="./Models",is_decoder = True).to(device)

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def tokenize_code(code_snippet, tokenizer):
    return tokenizer(
        code_snippet, max_length=512, truncation=True, padding='max_length', return_tensors="pt"
    )

In [7]:
def preprocess_data(batch, tokenizer):
    batch["input_ids"] = tokenize_code(batch["func_code_string"], tokenizer)["input_ids"]
    batch["labels"] = tokenize_code(batch["func_documentation_string"], tokenizer)["input_ids"]
    return batch

In [8]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(200)).map(lambda x: preprocess_data(x, tokenizer), batched=True)

In [9]:
model

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [10]:
training_args = TrainingArguments(
    output_dir="./codebert-finetuned-roberta/results",    # output directory
    num_train_epochs=10,                   # number of epochs
    per_device_train_batch_size=16,       # batch size
    per_device_eval_batch_size=16,        # eval batch size
    warmup_steps=500,                     # warmup steps
    weight_decay=0.01,                    # weight decay
    logging_dir="./logs",                 # logging directory
    logging_steps=10,
    evaluation_strategy="steps",          # Evaluation after every logging step
    save_total_limit=2,                   # Keep only last two checkpoints
    save_steps=500,                       # Save model every 500 steps
    report_to="none",
    fp16=True                     # No reports (e.g., to wandb)
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

  0%|          | 0/630 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  2%|▏         | 10/630 [04:19<4:29:50, 26.11s/it]

{'loss': 18.2138, 'grad_norm': 63.97089767456055, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.16}


                                                  
  2%|▏         | 10/630 [05:57<4:29:50, 26.11s/it]

{'eval_loss': 18.06072998046875, 'eval_runtime': 98.6642, 'eval_samples_per_second': 2.027, 'eval_steps_per_second': 0.132, 'epoch': 0.16}


  3%|▎         | 20/630 [10:19<4:38:02, 27.35s/it]

...

 62%|██████▏   | 390/630 [3:56:23<1:56:45, 29.19s/it]

{'loss': 1.0224, 'grad_norm': 2.3515713214874268, 'learning_rate': 3.8700000000000006e-05, 'epoch': 6.19}


                                                     
 62%|██████▏   | 390/630 [3:58:09<1:56:45, 29.19s/it]

{'eval_loss': 1.4271224737167358, 'eval_runtime': 105.7196, 'eval_samples_per_second': 1.892, 'eval_steps_per_second': 0.123, 'epoch': 6.19}


 62%|██████▏   | 391/630 [3:58:37<4:01:28, 60.62s/it]

In [ ]:
model.save_pretrained("./codebert-finetuned-roberta")
tokenizer.save_pretrained("./codebert-finetuned-roberta")

In [13]:
def generate_documentation(code_snippet, model, tokenizer):
    # Tokenize input
    inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Generate output
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)
    
    # Decode generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [16]:
code_example = """
def add_numbers(a, b):
    return a + c
"""

In [ ]:
documentation = generate_documentation(code_example, model, tokenizer)
print("Generated Documentation:", documentation)